### Load Libraries

In [1]:
import numpy as np
import pandas as pd
import os, cv2, math
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

### Loading Trained Model

In [2]:
model_path_name='snap_nodule_dector_wts.h5'
# load retinanet model
model = models.load_model(model_path_name, backbone_name='resnet50')
model = models.convert_model(model)
labels_to_names = {0.0: 'N'}

Using TensorFlow backend.
















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





### Prediction on the images

In [3]:
# image location
data_location = '../sample_images'
imagefile=os.listdir(data_location)

In [4]:
# output location : images with predicted nodules
directory='../sample_outputs/nodules'
if not os.path.exists(directory):
    os.makedirs(directory)

In [5]:
retinanet_input = 512
patchsize = 256
class_th = 0.5
import pandas as pd
for file in imagefile:
    image_path =data_location +'/' + file
    imagename=file.split('/')[-1]
    imagefull = read_image_bgr(image_path)
    h=math.ceil(imagefull.shape[0]/patchsize)*patchsize-imagefull.shape[0]
    w=math.ceil(imagefull.shape[1]/patchsize)*patchsize-imagefull.shape[1]
    imagefull= cv2.copyMakeBorder(imagefull,0,h,0,w,cv2.BORDER_CONSTANT,value=[0,0,0])
    I=np.zeros(imagefull.shape)
    
    number_nodules=0
    dicts = {'xc':[], 'yc':[],'area':[],'box_label':[]}
    
    # make and loop over the patches
    for row in range(patchsize,imagefull.shape[0]+1,patchsize):
        for col in range(patchsize,imagefull.shape[1]+1,patchsize):
            image=imagefull[row-patchsize:row,col-patchsize:col,:]
            draw = image.copy()
            
            # preprocess image for network
            image = preprocess_image(image)
            image, scale = resize_image(image,retinanet_input)
            draw, scale2 = resize_image(draw,patchsize)
            
            # process image
            boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
            
            # correct for image scale
            boxes /= scale
                        
            for box, score, label in zip(boxes[0], scores[0], labels[0]):
                x1=box[0]+col-patchsize
                y1=box[1]+row-patchsize
                x2=box[2]+col-patchsize
                y2=box[3]+row-patchsize
                if score < class_th:
                    break
                number_nodules=number_nodules+1
                
                dicts['xc'].append((x1+x2)/2)
                dicts['yc'].append((y1+y2)/2)
                dicts['area'].append((box[2] - box[0] + 1) * (box[3] - box[1] + 1))
                dicts['box_label'].append(box)
    
                color = label_color(label)
                b = box.astype(int)
                draw_box(draw, b, color=color)
                
                caption = "{} {:.3f}".format(labels_to_names[label], score)
                draw_caption(draw, b, caption)
            I[row-patchsize:row,col-patchsize:col,:]=draw
    
    cv2.imwrite(directory+'/'+imagename,I)
    print('number of nodules',number_nodules)
    df=pd.DataFrame.from_dict(dicts)
    df.to_csv(directory+'/'+imagename[:-4]+'.csv')

number of nodules 68


KeyboardInterrupt: 